#  天池新人实战赛o2o优惠券使用预测

> 

# 0. 导入包

In [1]:
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import SGDClassifier
import os, sys, pickle
import datetime
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys

print(sys.version)

3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]


# 1. EDA 

### 1.1 读入数据

In [3]:
# 读取数据
df_offline = pd.read_csv("../ali-data/ccf_offline_stage1_train.csv")
df_online = pd.read_csv("../ali-data/ccf_online_stage1_train.csv")
df_test = pd.read_csv("../ali-data/ccf_offline_stage1_test_revised.csv")

# offline 文件
df_offline.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN


### 1.2 赛题分析

    User_id        用户id

    Merchant_id     商家id

    Coupon_id	    优惠券id

    Discount_rate	  满x减y

    Distance	    x*500米,user常活动的地点与店铺距离，0：小于500米，10：大于5公里 

    Date_received  	领劵日期

    Date          消费日期
    
> 预测15天内使用优惠卷的概率。

### 1.3 样本数量分布

In [4]:
print("样本总数：",df_offline.shape[0])
print("有卷，且使用:",df_offline[pd.notnull(df_offline["Coupon_id"]) & pd.notnull(df_offline["Date"])].shape[0])

print("有卷，未使用:",df_offline[pd.notnull(df_offline["Coupon_id"]) & pd.isnull(df_offline["Date"])].shape[0])

print("无卷，且购买:",df_offline[pd.isnull(df_offline["Coupon_id"]) & pd.notnull(df_offline["Date"])].shape[0])

print("无卷，未购买:",df_offline[pd.isnull(df_offline["Coupon_id"]) & pd.isnull(df_offline["Date"])].shape[0])

样本总数： 1754884
有卷，且使用: 75382
有卷，未使用: 977900
无卷，且购买: 701602
无卷，未购买: 0


In [5]:
print("测试集")
print("     - 样本数量: ", df_test.shape[0])
print("     - 不重复的用户数量：" ,len(set(df_test["User_id"])))
print("     - 不重复的商家数量：" ,len(set(df_test["Merchant_id"])))

print("训练集")
print("     - 样本数量: ", df_offline.shape[0])
print("     - 不重复的用户数量：" ,len(set(df_offline["User_id"])))
print("     - 不重复的商家数量：" ,len(set(df_offline["Merchant_id"])))

print("同时出现在训练集，测试集中的用户数量：", len(set(df_test["User_id"]) & set(set(df_offline["User_id"]))))


测试集
     - 样本数量:  113640
     - 不重复的用户数量： 76309
     - 不重复的商家数量： 1559
训练集
     - 样本数量:  1754884
     - 不重复的用户数量： 539438
     - 不重复的商家数量： 8415
同时出现在训练集，测试集中的用户数量： 76307


### 1.4 简单分析

1. train & test 有重复出现的用户。
    
       可以针对用户的消费习惯（消费能力，消费倾向的商店，是否喜欢使用优惠券等等）深入挖掘；

2. 存在大量的用户领卷未使用。
        
       为什么领券未使用？领券时间？距离？优惠幅度？优惠类型？
       
3. 未领券，且购买用户也很多，但测试集中只存在领卷的用户。
        
       未领券用户是否也参与用户消费习惯的特征挖掘呢？
       
4. 商家也存在大量重复现象。

       可以针对商家的店铺吸引力（是否有常客，常客使用优惠券比例，店铺活动力度/频率， 客户优惠券使用率，）深入挖掘；
       
5. 线上数据。
   
       线上数据可以挖掘用户消费习惯，店铺吸引力。

# 2. 特征工程

### 2.1  优惠券特征

In [6]:
# 折扣率
def discount_rate(x):
    x = str(x)
    length = len(x)
    a=1
    b=1
    try:
        if length == 3:
            return 1.0
        elif ":" in x:
            a,b = x.split(":")
            return 1-float(b)/float(a)
    except:
        return x

# 周几
def date2week(row):
    if pd.isnull(row):
        return 0
    date = pd.to_datetime(row,format="%Y%m%d")
    return date.weekday()+1


## 类别特征-是否是周五，周六，周日
def isOffDay(row):
    try:
        if row in [5,6,7]:
            return 1
        return 0
    except:
        print("error")
        
##  优惠券类型
def discount_cls(x):
    x = str(x)
    length = len(x)
    if length == 3:
        return 3
    elif ":" in x:
        return 1
    else:
        return 2

    
# 优惠券特征
def getCouponFeature(dataset):

    # 折扣率
    dataset["fCoupon_discountRate"] = dataset.Discount_rate.apply(discount_rate)
    
    # 是否是节假日
    dataset["week"] = dataset.Date_received.apply(date2week)
    dataset["fCoupon_isOffDay"] = dataset.week.apply(isOffDay)
    
    # 优惠类型    
    dataset["Discount_cls"] = dataset["Discount_rate"].copy()
    dataset["Discount_cls"] = dataset.Discount_cls.apply(discount_cls)
    dataset[["fCoupon_cls_jian", "fCoupon_cls_discount","fCoupon_cls_None"]]=pd.get_dummies(dataset["Discount_cls"])

    
    # 优惠券历史信息
    return dataset

### 2.2 商家特征

In [7]:
def getMerchanFeature(dataset, feature):
 
    merchant = feature[['Merchant_id','Coupon_id','Distance','Date_received','Date']].copy()
#     g = merchant[['merchant_id']].copy() 
    
    # 商家产生的消费总次数
    tmp = merchant[pd.notnull(merchant["Date"])].copy()
    g1 = tmp.groupby("Merchant_id").size().reset_index(name="fMerchant_consumptionCount")
    
    # 消费用优惠券的次数
    tmp = merchant[(pd.notnull(merchant["Date"])) & pd.notnull(merchant["Date_received"])].copy()
    g2 = tmp.groupby("Merchant_id").size().reset_index(name="fMerchant_useCouponConsumptionCount")
    
    # 消费使用优惠券的比例
    g3 = pd.merge(g1,g2,on='Merchant_id',how='left')
    g3["fMerchant_consumptionUseCouponRate"] = (g3["fMerchant_useCouponConsumptionCount"].astype('float'))/(g3["fMerchant_consumptionCount"].astype('float'))
    g3.drop(["fMerchant_useCouponConsumptionCount", "fMerchant_consumptionCount"], axis=1,inplace = True)
   
    # 发放的优惠券的数量
    tmp = merchant[pd.notnull(merchant["Date_received"])].copy()
    g4 = tmp.groupby("Merchant_id").size().reset_index(name="fMerchant_sendCouponCount")
    
    # 优惠券核销率
    g5 = pd.merge(g4,g2,on='Merchant_id',how='left')
    g5["fMerchant_sendUseCouponRate"] = (g5["fMerchant_useCouponConsumptionCount"].astype('float'))/(g5["fMerchant_sendCouponCount"].astype('float'))
    g5.drop(["fMerchant_sendCouponCount", "fMerchant_useCouponConsumptionCount"], axis=1,inplace = True)

    dataset = pd.merge(dataset, g1, on="Merchant_id", how="left")
    dataset = pd.merge(dataset, g2, on="Merchant_id", how="left")
    dataset = pd.merge(dataset, g3, on="Merchant_id", how="left")
    dataset = pd.merge(dataset, g4, on="Merchant_id", how="left")
    dataset = pd.merge(dataset, g5, on="Merchant_id", how="left")
    
    return dataset

### 2.3 根据时间划分数据集和特征来源

In [8]:
# 合并
data = pd.concat([df_offline, df_test], keys=(["train", "test"]))


"""
    数据集 | 特征来源 |  最后样本
    ---   | ---     |  ---
    train | 2.14～5.14 | 5.15～6.15
    test  | 4.1～6.30 | 7.1～7.30
"""

train_data = data[(pd.notnull(data["Date_received"])) & (data["Date_received"] >=20160515)
                      &(data["Date_received"] <= 20160615)]

train_feature = data[(pd.notnull(data["Date_received"])) & (data["Date_received"] >=20160214)
                      &(data["Date_received"] <= 20160514)]


test_data = data[(pd.notnull(data["Date_received"])) & (data["Date_received"] >=20160701)
                      &(data["Date_received"] <= 20160730)]

test_feature = data[(pd.notnull(data["Date_received"])) & (data["Date_received"] >=20160401)
                      &(data["Date_received"] <= 20160630)]


In [12]:
# 处理标签
def makeLabel(row):
    if pd.isnull(row["Date_received"]) : # and pd.isna(row["Date"]) == False:
        return -1
    elif row["Date"] != "null" :
        res = pd.to_datetime(row["Date"],format="%Y%m%d") - pd.to_datetime(row["Date_received"],format="%Y%m%d")
        if res <= pd.Timedelta(15,"D"):
            return 1
    return 0

In [13]:
def dataProcess(train_data, train_feature, test_data, test_feature):

    train_data = getCouponFeature(train_data)
    test_data = getCouponFeature(test_data)

    train_data = getMerchanFeature(train_data, train_feature)
    test_data = getMerchanFeature(test_data, test_feature)
    
    train_data.drop_duplicates(inplace=True)
    

    
    
    train_data["label"] = train_data.apply(makeLabel, axis=1)   
    test_data["label"] = test_data.apply(makeLabel, axis=1)
    
#     train_data = train_data.drop(
#     columns=['User_id', 'Merchant_id', 'Discount_rate', 'Date_received',
#              'Date', 'Coupon_id'], axis=1)
#     test_data = test_data.drop(
#         columns=['User_id', 'Merchant_id', 'Discount_rate', 'Date_received','Date',
#                  'Coupon_id'

#                 ], axis=1)
    
        # 处理缺失值
    train_data = train_data.fillna(0)
    test_data = test_data.fillna(0)
    
    return train_data, test_data

In [14]:
train_data, test_data = dataProcess(train_data, train_feature, test_data, test_feature)

In [15]:
train_data.head()

,Coupon_id,Date,Date_received,Discount_rate,Distance,Merchant_id,User_id,fCoupon_discountRate,week,fCoupon_isOffDay,Discount_cls,fCoupon_cls_jian,fCoupon_cls_discount,fCoupon_cls_None,fMerchant_consumptionCount,fMerchant_useCouponConsumptionCount,fMerchant_consumptionUseCouponRate,fMerchant_sendCouponCount,fMerchant_sendUseCouponRate,label
0,11002.0,0.0,20160528.0,150:20,1.0,4663,1439408,0.866667,6,1,1,1,0,0,9.0,9.0,1.0,595.0,0.015126,0
1,8591.0,0.0,20160613.0,20:1,0.0,2632,1439408,0.950000,1,0,1,1,0,0,2.0,2.0,1.0,27.0,0.074074,0
2,8591.0,20160613.0,20160516.0,20:1,0.0,2632,1439408,0.950000,1,0,1,1,0,0,2.0,2.0,1.0,27.0,0.074074,0
3,1532.0,0.0,20160530.0,30:5,0.0,450,2029232,0.833333,1,0,1,1,0,0,626.0,626.0,1.0,15894.0,0.039386,0
4,12737.0,0.0,20160519.0,20:1,0.0,6459,2029232,0.950000,4,0,1,1,0,0,0.0,0.0,0.0,0.0,0.000000,0


### 2.4 样本均衡

In [16]:
train_data.groupby("label").size()

label
0    233996
1     23130
dtype: int64

In [17]:
train_0 = train_data.groupby("label").get_group(0)[:23130]

train_1 = train_data.groupby("label").get_group(1)

frams = [train_0, train_1]
train_data = pd.concat(frams)


In [18]:
train_data.to_csv("../ali-data/train_data.csv")

test_data.to_csv("../ali-data/test_data.csv")

### 2.5 断点

In [19]:
train_data = pd.read_csv("../ali-data/train_data.csv")
test_data = pd.read_csv("../ali-data/test_data.csv")

train_data.head()
train_data.drop(["Unnamed: 0"],axis=1,inplace=True)
test_data.drop(["Unnamed: 0"],axis=1,inplace=True)

train_data.head()

,Coupon_id,Date,Date_received,Discount_rate,Distance,Merchant_id,User_id,fCoupon_discountRate,week,fCoupon_isOffDay,Discount_cls,fCoupon_cls_jian,fCoupon_cls_discount,fCoupon_cls_None,fMerchant_consumptionCount,fMerchant_useCouponConsumptionCount,fMerchant_consumptionUseCouponRate,fMerchant_sendCouponCount,fMerchant_sendUseCouponRate,label
0,11002.0,0.0,20160528.0,150:20,1.0,4663,1439408,0.866667,6,1,1,1,0,0,9.0,9.0,1.0,595.0,0.015126,0
1,8591.0,0.0,20160613.0,20:1,0.0,2632,1439408,0.950000,1,0,1,1,0,0,2.0,2.0,1.0,27.0,0.074074,0
2,8591.0,20160613.0,20160516.0,20:1,0.0,2632,1439408,0.950000,1,0,1,1,0,0,2.0,2.0,1.0,27.0,0.074074,0
3,1532.0,0.0,20160530.0,30:5,0.0,450,2029232,0.833333,1,0,1,1,0,0,626.0,626.0,1.0,15894.0,0.039386,0
4,12737.0,0.0,20160519.0,20:1,0.0,6459,2029232,0.950000,4,0,1,1,0,0,0.0,0.0,0.0,0.0,0.000000,0


In [20]:
# 性能评价函数
def myauc(test):
    testgroup = test.groupby(['Coupon_id'])
    aucs = []
    for i in testgroup:
        tmpdf = i[1]
        if len(tmpdf['label'].unique()) != 2:
            continue
        fpr, tpr, thresholds = roc_curve(tmpdf['label'], tmpdf['pred'], pos_label=1)
        aucs.append(auc(fpr, tpr))
    return np.average(aucs)

In [37]:
# import datetime
import os
import time
from concurrent.futures import ProcessPoolExecutor
from math import ceil

#from catboost import CatBoostClassifier
#from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.model_selection import  train_test_split

# feature_list = ["f_off_day","f_discount_rate","f_discount_cls","f_cls_jian","f_cls_discount","f_cls_None",
#       "f_coupon_no_used","f_no_coupon","f_coupon_used_rate","f_consum_used_coupon_rate","Distance"]
def train_xgb(train_data, test_data):
    predict_dataset = test_data[['User_id', 'Coupon_id', 'Date_received']].copy()

    # 将数据转化为dmatric格式
    train_data_x = train_data.drop(
        columns=['User_id', 'Merchant_id', 'Discount_rate', 'Date_received',"Discount_rate",
                 'Date', 'Coupon_id', 'label'], axis=1)
    test_data_x = test_data.drop(
        columns=['User_id', 'Merchant_id', 'Discount_rate', 'Date_received','Date',"Discount_rate",
                 'Coupon_id','label'

                ], axis=1) 
    
    x_train, x_valid, y_train, y_valid = train_test_split(train_data_x, train_data["label"],
                                                      test_size=0.2, random_state=66)

    train_dmatrix = xgb.DMatrix(x_train, label=y_train)
    
    valid_dmatrix = xgb.DMatrix(x_valid, label=y_valid)
    
    evallist = [(train_dmatrix,"train"),(valid_dmatrix,"valid")]
    
    predict_dmatrix = xgb.DMatrix(test_data_x)

    # xgboost模型训练
    params = {
#         'booster':'gbtree',
#         'objective': 'binary:logistic',
#         'eval_metric':'auc',
#         'gamma':0.1,
#         'min_child_weight':1,
#         'max_depth':10,
#         'lambda':10,
#         'subsample':1,
#         'colsample_bytree': 1,
#         'colsample_bylevel':1,
#         'eta': 0.1,
#         'tree_method':'exact',
#         'seed':0,
#         'nthread':12,
        
        'booster':'gbtree',
        'objective': 'binary:logistic',
        'eval_metric':'auc',
        'gamma':0.1,
        'min_child_weight':1,
        'max_depth':8,
        'lambda':10,
        'subsample':1,
        'colsample_bytree': 1,
        'colsample_bylevel':1,
        'eta': 0.1,
        'tree_method':'exact',
        'seed':0,
        'nthread':12,

    }

    # 使用xgb.cv优化num_boost_round参数
    cvresult = xgb.cv(params, train_dmatrix, num_boost_round=1000, nfold=10, metrics='auc', seed=0, callbacks=[
        xgb.callback.print_evaluation(show_stdv=False),
        xgb.callback.early_stop(50)
    ])
    num_round_best = cvresult.shape[0] - 1
    print('Best round num: ', num_round_best)

    # 使用优化后的num_boost_round参数训练模型
    watchlist = [(train_dmatrix, 'train')]
    model = xgb.train(params, train_dmatrix, num_boost_round=num_round_best,  evals=evallist)
    #model = xgb.train(params, train_dmatrix, 1000,  evals=evallist, early_stopping_rounds=30)

    model.save_model('./1xgbmodel')
    params['predictor'] = 'cpu_predictor'
    model = xgb.Booster(params)
    model.load_model('./1xgbmodel')

    # predict test set
    test_data_predict = predict_dataset.copy()
    test_data_predict['label'] = model.predict(predict_dmatrix)

    # 标签归一化
    test_data_predict.label = MinMaxScaler(copy=True, feature_range=(0, 1)).fit_transform(
        test_data_predict.label.values.reshape(-1, 1))
    test_data_predict.sort_values(by=['Coupon_id', 'label'], inplace=True)
    
    test_data_predict[["User_id", 'Coupon_id','Date_received']] =  test_data_predict[["User_id", 'Coupon_id','Date_received']].astype(int)
    test_data_predict.to_csv("./xgb_preds.csv", index=None, header=None)
    print(test_data_predict.describe())

    # 在dataset12上计算auc
    model = xgb.Booster()
    model.load_model('./1xgbmodel')

    temp = train_data[['Coupon_id', 'label']].copy()
    temp['pred'] = model.predict(xgb.DMatrix(train_data_x))
    temp.pred = MinMaxScaler(copy=True, feature_range=(0, 1)).fit_transform(temp['pred'].values.reshape(-1, 1))
    print(myauc(temp))
    
train_xgb(train_data, test_data)

[0]	train-auc:0.8098	test-auc:0.801316
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 50 rounds.
[1]	train-auc:0.810116	test-auc:0.801545
[2]	train-auc:0.810331	test-auc:0.801875
[3]	train-auc:0.810371	test-auc:0.801898
[4]	train-auc:0.810488	test-auc:0.801938
[5]	train-auc:0.810814	test-auc:0.802458
[6]	train-auc:0.811019	test-auc:0.802647
[7]	train-auc:0.811168	test-auc:0.802863
[8]	train-auc:0.811288	test-auc:0.802845
[9]	train-auc:0.811458	test-auc:0.803107
[10]	train-auc:0.811661	test-auc:0.803295
[11]	train-auc:0.811904	test-auc:0.803528
[12]	train-auc:0.812099	test-auc:0.803732
[13]	train-auc:0.812292	test-auc:0.803853
[14]	train-auc:0.81256	test-auc:0.804143
[15]	train-auc:0.812866	test-auc:0.804371
[16]	train-auc:0.813166	test-auc:0.804629
[17]	train-auc:0.813437	test-auc:0.804831
[18]	train-auc:0.81379	test-auc:0.805172
[19]	train-auc:0.814009	test-auc:0.805364
[20]	train-auc:0.814409	test-auc:

[191]	train-auc:0.829852	test-auc:0.815966
[192]	train-auc:0.829914	test-auc:0.815989
[193]	train-auc:0.829981	test-auc:0.816018
[194]	train-auc:0.830048	test-auc:0.816043
[195]	train-auc:0.830101	test-auc:0.816056
[196]	train-auc:0.830162	test-auc:0.816079
[197]	train-auc:0.830221	test-auc:0.816111
[198]	train-auc:0.830283	test-auc:0.816139
[199]	train-auc:0.830346	test-auc:0.816152
[200]	train-auc:0.830396	test-auc:0.816152
[201]	train-auc:0.830454	test-auc:0.816161
[202]	train-auc:0.830506	test-auc:0.816169
[203]	train-auc:0.83057	test-auc:0.816171
[204]	train-auc:0.830622	test-auc:0.816204
[205]	train-auc:0.830674	test-auc:0.81622
[206]	train-auc:0.830733	test-auc:0.81623
[207]	train-auc:0.830795	test-auc:0.816255
[208]	train-auc:0.830855	test-auc:0.816278
[209]	train-auc:0.830906	test-auc:0.816281
[210]	train-auc:0.830965	test-auc:0.816321
[211]	train-auc:0.831016	test-auc:0.816334
[212]	train-auc:0.831081	test-auc:0.816345
[213]	train-auc:0.831128	test-auc:0.816355
[214]	train-au

[383]	train-auc:0.838122	test-auc:0.818774
[384]	train-auc:0.838154	test-auc:0.818784
[385]	train-auc:0.838178	test-auc:0.81879
[386]	train-auc:0.838203	test-auc:0.818794
[387]	train-auc:0.838236	test-auc:0.818805
[388]	train-auc:0.838261	test-auc:0.81882
[389]	train-auc:0.838284	test-auc:0.81883
[390]	train-auc:0.838307	test-auc:0.818848
[391]	train-auc:0.83833	test-auc:0.81885
[392]	train-auc:0.838355	test-auc:0.818863
[393]	train-auc:0.83838	test-auc:0.818875
[394]	train-auc:0.8384	test-auc:0.818884
[395]	train-auc:0.838424	test-auc:0.818886
[396]	train-auc:0.838447	test-auc:0.818888
[397]	train-auc:0.838475	test-auc:0.818894
[398]	train-auc:0.838496	test-auc:0.818894
[399]	train-auc:0.838518	test-auc:0.818903
[400]	train-auc:0.838548	test-auc:0.81892
[401]	train-auc:0.838573	test-auc:0.818928
[402]	train-auc:0.838594	test-auc:0.818933
[403]	train-auc:0.838621	test-auc:0.818943
[404]	train-auc:0.838645	test-auc:0.818945
[405]	train-auc:0.838673	test-auc:0.818947
[406]	train-auc:0.83

[575]	train-auc:0.842204	test-auc:0.819511
[576]	train-auc:0.842224	test-auc:0.819511
[577]	train-auc:0.842243	test-auc:0.819512
[578]	train-auc:0.842261	test-auc:0.819508
[579]	train-auc:0.842273	test-auc:0.81951
[580]	train-auc:0.842295	test-auc:0.819506
[581]	train-auc:0.842309	test-auc:0.819506
[582]	train-auc:0.842323	test-auc:0.819504
[583]	train-auc:0.842339	test-auc:0.819504
[584]	train-auc:0.842358	test-auc:0.819501
[585]	train-auc:0.842379	test-auc:0.819503
[586]	train-auc:0.842397	test-auc:0.819503
[587]	train-auc:0.84242	test-auc:0.819509
[588]	train-auc:0.842438	test-auc:0.819512
[589]	train-auc:0.842458	test-auc:0.819513
[590]	train-auc:0.842478	test-auc:0.819523
[591]	train-auc:0.842498	test-auc:0.819518
[592]	train-auc:0.842517	test-auc:0.819516
[593]	train-auc:0.842536	test-auc:0.819515
[594]	train-auc:0.842552	test-auc:0.81952
[595]	train-auc:0.842566	test-auc:0.819521
[596]	train-auc:0.842585	test-auc:0.819521
[597]	train-auc:0.842602	test-auc:0.819526
[598]	train-au

[767]	train-auc:0.84562	test-auc:0.819835
[768]	train-auc:0.845634	test-auc:0.819839
[769]	train-auc:0.845645	test-auc:0.819843
[770]	train-auc:0.845658	test-auc:0.81984
[771]	train-auc:0.845672	test-auc:0.819843
[772]	train-auc:0.845685	test-auc:0.819839
[773]	train-auc:0.845705	test-auc:0.819843
[774]	train-auc:0.845715	test-auc:0.819845
[775]	train-auc:0.845729	test-auc:0.819846
[776]	train-auc:0.845747	test-auc:0.819846
[777]	train-auc:0.845762	test-auc:0.819847
[778]	train-auc:0.845776	test-auc:0.819842
[779]	train-auc:0.845792	test-auc:0.819839
[780]	train-auc:0.845804	test-auc:0.819832
[781]	train-auc:0.845821	test-auc:0.819833
[782]	train-auc:0.845835	test-auc:0.819836
[783]	train-auc:0.845853	test-auc:0.819833
[784]	train-auc:0.845866	test-auc:0.819835
[785]	train-auc:0.845879	test-auc:0.819835
[786]	train-auc:0.845892	test-auc:0.819834
[787]	train-auc:0.845914	test-auc:0.81983
[788]	train-auc:0.845928	test-auc:0.81983
[789]	train-auc:0.845948	test-auc:0.819831
[790]	train-auc

...

[150]	train-auc:0.827004	valid-auc:0.816531
[151]	train-auc:0.827064	valid-auc:0.816558
[152]	train-auc:0.827121	valid-auc:0.816553
[153]	train-auc:0.827182	valid-auc:0.81656
[154]	train-auc:0.827263	valid-auc:0.816624
[155]	train-auc:0.827322	valid-auc:0.816674
[156]	train-auc:0.827374	valid-auc:0.816721
[157]	train-auc:0.827415	valid-auc:0.816763
[158]	train-auc:0.827461	valid-auc:0.816754
[159]	train-auc:0.827534	valid-auc:0.816771
[160]	train-auc:0.827654	valid-auc:0.816825
[161]	train-auc:0.827745	valid-auc:0.816893
[162]	train-auc:0.827794	valid-auc:0.816931
[163]	train-auc:0.827871	valid-auc:0.817008
[164]	train-auc:0.827957	valid-auc:0.817077
[165]	train-auc:0.828013	valid-auc:0.817137
[166]	train-auc:0.828069	valid-auc:0.817172
[167]	train-auc:0.828158	valid-auc:0.817218
[168]	train-auc:0.828226	valid-auc:0.817245
[169]	train-auc:0.828303	valid-auc:0.817255
[170]	train-auc:0.828399	valid-auc:0.817301
[171]	train-auc:0.828487	valid-auc:0.81736
[172]	train-auc:0.828535	valid-auc

[338]	train-auc:0.836066	valid-auc:0.82033
[339]	train-auc:0.836087	valid-auc:0.820322
[340]	train-auc:0.836116	valid-auc:0.820347
[341]	train-auc:0.836139	valid-auc:0.820322
[342]	train-auc:0.836161	valid-auc:0.820362
[343]	train-auc:0.836189	valid-auc:0.820368
[344]	train-auc:0.836259	valid-auc:0.820417
[345]	train-auc:0.836309	valid-auc:0.820455
[346]	train-auc:0.836347	valid-auc:0.820489
[347]	train-auc:0.83639	valid-auc:0.820518
[348]	train-auc:0.836423	valid-auc:0.820544
[349]	train-auc:0.836475	valid-auc:0.820607
[350]	train-auc:0.836524	valid-auc:0.820608
[351]	train-auc:0.836562	valid-auc:0.820627
[352]	train-auc:0.83659	valid-auc:0.82064
[353]	train-auc:0.836622	valid-auc:0.820675
[354]	train-auc:0.836649	valid-auc:0.820694
[355]	train-auc:0.836675	valid-auc:0.820717
[356]	train-auc:0.836703	valid-auc:0.820724
[357]	train-auc:0.836738	valid-auc:0.820754
[358]	train-auc:0.836771	valid-auc:0.820763
[359]	train-auc:0.83681	valid-auc:0.820778
[360]	train-auc:0.836842	valid-auc:0.

[526]	train-auc:0.840501	valid-auc:0.822573
[527]	train-auc:0.840508	valid-auc:0.822581
[528]	train-auc:0.840541	valid-auc:0.822594
[529]	train-auc:0.840551	valid-auc:0.822598
[530]	train-auc:0.840592	valid-auc:0.822603
[531]	train-auc:0.840604	valid-auc:0.82261
[532]	train-auc:0.840629	valid-auc:0.822615
[533]	train-auc:0.840642	valid-auc:0.822616
[534]	train-auc:0.840656	valid-auc:0.822616
[535]	train-auc:0.840669	valid-auc:0.82262
[536]	train-auc:0.840684	valid-auc:0.822616
[537]	train-auc:0.840711	valid-auc:0.822626
[538]	train-auc:0.840725	valid-auc:0.82263
[539]	train-auc:0.84074	valid-auc:0.822629
[540]	train-auc:0.840763	valid-auc:0.822636
[541]	train-auc:0.840777	valid-auc:0.82263
[542]	train-auc:0.84079	valid-auc:0.822643
[543]	train-auc:0.840816	valid-auc:0.822642
[544]	train-auc:0.840825	valid-auc:0.822649
[545]	train-auc:0.840862	valid-auc:0.822666
[546]	train-auc:0.840877	valid-auc:0.822669
[547]	train-auc:0.840892	valid-auc:0.822679
[548]	train-auc:0.840914	valid-auc:0.8

[714]	train-auc:0.844067	valid-auc:0.823121
[715]	train-auc:0.844076	valid-auc:0.823118
[716]	train-auc:0.844088	valid-auc:0.823118
[717]	train-auc:0.844096	valid-auc:0.823124
[718]	train-auc:0.844115	valid-auc:0.823111
[719]	train-auc:0.84413	valid-auc:0.823104
[720]	train-auc:0.844152	valid-auc:0.823118
[721]	train-auc:0.844158	valid-auc:0.823119
[722]	train-auc:0.844174	valid-auc:0.823124
[723]	train-auc:0.844188	valid-auc:0.823112
[724]	train-auc:0.844201	valid-auc:0.823121
[725]	train-auc:0.844216	valid-auc:0.823115
[726]	train-auc:0.844229	valid-auc:0.823117
[727]	train-auc:0.844238	valid-auc:0.823115
[728]	train-auc:0.844253	valid-auc:0.823115
[729]	train-auc:0.844269	valid-auc:0.823122
[730]	train-auc:0.844274	valid-auc:0.823127
[731]	train-auc:0.844286	valid-auc:0.823127
[732]	train-auc:0.844308	valid-auc:0.82312
[733]	train-auc:0.844322	valid-auc:0.823127
[734]	train-auc:0.844326	valid-auc:0.823131
[735]	train-auc:0.844367	valid-auc:0.82315
[736]	train-auc:0.844379	valid-auc:

[902]	train-auc:0.847277	valid-auc:0.823453
[903]	train-auc:0.847292	valid-auc:0.823448
[904]	train-auc:0.847299	valid-auc:0.823437
[905]	train-auc:0.847304	valid-auc:0.823438
[906]	train-auc:0.84731	valid-auc:0.823443
[907]	train-auc:0.847344	valid-auc:0.823439
[908]	train-auc:0.847351	valid-auc:0.823435
[909]	train-auc:0.847377	valid-auc:0.823441
[910]	train-auc:0.847406	valid-auc:0.823433
[911]	train-auc:0.847413	valid-auc:0.82343
[912]	train-auc:0.847444	valid-auc:0.823431
[913]	train-auc:0.847452	valid-auc:0.82343
[914]	train-auc:0.847476	valid-auc:0.823423
[915]	train-auc:0.847491	valid-auc:0.823427
[916]	train-auc:0.847499	valid-auc:0.823414
[917]	train-auc:0.847529	valid-auc:0.82343
[918]	train-auc:0.847562	valid-auc:0.823448
[919]	train-auc:0.847573	valid-auc:0.823455
[920]	train-auc:0.847604	valid-auc:0.823462
[921]	train-auc:0.847611	valid-auc:0.823461
[922]	train-auc:0.847639	valid-auc:0.82347
[923]	train-auc:0.847648	valid-auc:0.823466
[924]	train-auc:0.847677	valid-auc:0.

In [33]:
from sklearn.model_selection import GridSearchCV

In [ ]:

cv_params = {'n_estimators': [400, 500, 600, 700, 800]}
other_params = {'learning_rate': 0.1, 'n_estimators': 500, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,
                'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

model = xgb.XGBRegressor(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='r2', cv=5, verbose=1, n_jobs=4)
optimized_GBM.fit(X_train, y_train)
evalute_result = optimized_GBM.grid_scores_
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))